In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder \
    .appName("LoanDefaultPrediction") \
    .config("spark.hadoop.fs.s3a.access.key", "minio_access_key") \
    .config("spark.hadoop.fs.s3a.secret.key", "minio_secret_key") \
    .config("spark.hadoop.fs.s3a.endpoint", "http://localhost:9000")\
    .config("spark.jars", "/home/drissdo/Desktop/End-to-end-NYC-analysis/src/jar/aws-java-sdk-bundle-1.11.901.jar, /home/drissdo/Desktop/End-to-end-NYC-analysis/src/jar/hadoop-aws-3.3.1.jar")\
    .config("spark.hadoop.fs.s3a.path.style.access", "true")\
    .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")\
    .getOrCreate()

25/04/28 21:15:20 WARN Utils: Your hostname, dtdat resolves to a loopback address: 127.0.1.1; using 192.168.2.12 instead (on interface wlp0s20f3)
25/04/28 21:15:20 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
25/04/28 21:15:20 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


25/04/28 21:15:34 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


In [6]:
df = spark.read.parquet("s3a://silver/yellow_tripdata_2024-01/part-00001-0074987a-f48b-41e9-a464-bbf4648c7ab8-c000.snappy.parquet")

In [7]:
df.show()

+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|Airport_fee|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|       2| 2024-01-01 00:57:55|  2024-01-01 01:17:43|              1|         1.72|         1|                 N|         186|          79|           2|       17.7|  1.0|    0.5|       0.

In [ ]:
df.isna().sum()

AttributeError: 'DataFrame' object has no attribute 'isna'

25/04/09 10:53:00 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


In [ ]:
df

In [ ]:
df = df.fillna({
    "passenger_count": 1,              # assuming 1 passenger as a safe default
    "RatecodeID": 1,                   # default is Standard rate
    "store_and_fwd_flag": "N",         # default is No
    "congestion_surcharge": 0.0,       # assume no congestion surcharge
    "Airport_fee": 0.0                 # assume no airport fee
})

In [ ]:
df.isna().sum()

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("/home/drissdo/Desktop/End-to-end-NYC-analysis/src/notebook/yellow_tripdata_2024-01.csv")

In [3]:
df

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee
0,2,2024-01-01 00:57:55,2024-01-01 01:17:43,1,1.72,1,N,186,79,2,17.7,1.0,0.5,0.00,0.0,1.0,22.70,2.5,0.0
1,1,2024-01-01 00:03:00,2024-01-01 00:09:36,1,1.80,1,N,140,236,1,10.0,3.5,0.5,3.75,0.0,1.0,18.75,2.5,0.0
2,1,2024-01-01 00:17:06,2024-01-01 00:35:01,1,4.70,1,N,236,79,1,23.3,3.5,0.5,3.00,0.0,1.0,31.30,2.5,0.0
3,1,2024-01-01 00:36:38,2024-01-01 00:44:56,1,1.40,1,N,79,211,1,10.0,3.5,0.5,2.00,0.0,1.0,17.00,2.5,0.0
4,1,2024-01-01 00:46:51,2024-01-01 00:52:57,1,0.80,1,N,211,148,1,7.9,3.5,0.5,3.20,0.0,1.0,16.10,2.5,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2699908,2,2024-01-30 19:40:55,2024-01-30 19:53:02,1,3.14,1,N,161,238,1,15.6,2.5,0.5,4.42,0.0,1.0,26.52,2.5,0.0
2699909,2,2024-01-30 19:08:39,2024-01-30 19:16:31,1,1.45,1,N,163,246,1,10.0,2.5,0.5,3.00,0.0,1.0,19.50,2.5,0.0
2699910,2,2024-01-30 19:46:49,2024-01-30 19:59:02,1,2.97,1,N,230,236,1,14.9,2.5,0.5,3.21,0.0,1.0,24.61,2.5,0.0
2699911,1,2024-01-30 19:41:31,2024-01-30 19:46:23,0,0.80,1,N,170,107,1,6.5,5.0,0.5,8.00,0.0,1.0,21.00,2.5,0.0
